![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### Portfolio uchun vazifa: Toshkent shahrida uylarning narxini aniqlash.


In [100]:
import pandas as pd
imp
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


In [145]:
df = df.drop("location", axis=1)
df

,district,rooms,size,level,max_levels,price
0,Юнусабадский,3.0,57.0,4,4,52000
1,Яккасарайский,2.0,52.0,4,5,56000
2,Чиланзарский,2.0,42.0,4,4,37000
3,Чиланзарский,3.0,65.0,1,4,49500
4,Чиланзарский,3.0,70.0,3,5,55000
...,...,...,...,...,...,...
7560,Яшнободский,1.0,38.0,5,5,24500
7561,Яшнободский,2.0,49.0,1,4,32000
7562,Шайхантахурский,2.0,64.0,3,9,40000
7563,Мирзо-Улугбекский,1.0,18.0,1,4,11000


In [102]:
df.district.value_counts()

Чиланзарский         1641
Мирзо-Улугбекский    1382
Юнусабадский          883
Учтепинский           880
Мирабадский           773
Яккасарайский         658
Яшнободский           484
Шайхантахурский       396
Олмазорский           269
Сергелийский          175
Янгихаётский           16
Бектемирский            8
Name: district, dtype: int64

In [128]:
df['rooms'] = df.rooms.astype(float)

<ipython-input-128-d34a7805113c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rooms'] = df.rooms.astype(float)


In [133]:
df = df[df['size'] !='Площадьземли:1сот']

In [147]:
df['rooms'] = df['rooms'].astype(int)

In [148]:
df.dtypes

district       object
rooms           int64
size          float64
level           int64
max_levels      int64
price           int64
dtype: object

In [149]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_set.drop("price", axis=1)
y = train_set["price"].copy()

X_num = X_train.drop('district', axis=1)

In [164]:
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [165]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('std_scaler', StandardScaler())             
])

In [166]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['district']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [167]:
X_prepared = full_pipeline.fit_transform(X_train)

In [170]:
X_prepared[0:5,:]

array([[-0.57458889, -0.03523441,  0.12524471, -0.78114458,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [-1.4952878 , -0.05655365, -1.20092799, -1.54042117,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.26680894,  0.00882535, -0.31681286, -0.40150628,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.57458889, -0.03381313, -0.31681286,  1.11704692,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.57458889, -0.04091954, -1

In [171]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

In [172]:
LR_model.fit(X_prepared, y)

LinearRegression()

In [173]:
# tasodifiy 5 ta qatorni ajratib olamiz
test_data = X_train.sample(5)
test_data

,district,rooms,size,level,max_levels
4664,Яшнободский,2,55.0,2,5
6895,Учтепинский,5,120.0,3,5
6079,Учтепинский,2,50.0,4,4
2172,Мирзо-Улугбекский,2,45.0,2,4
7111,Чиланзарский,1,36.0,2,5


In [174]:
test_label = y.loc[test_data.index]
test_label

4664     30800
6895    120000
6079     29500
2172     33505
7111     25500
Name: price, dtype: int64

In [175]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

array([[-0.57458889, -0.03878762, -0.75887042, -0.40150628,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 2.18750785,  0.00740407, -0.31681286, -0.40150628,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.57458889, -0.04234083,  0.12524471, -0.78114458,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.57458889, -0.04589403, -0.75887042, -0.78114458,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-1.4952878 , -0.05228981, -0

In [176]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data

array([43803.50661514, 98108.86523535, 23549.37761107, 43537.24054253,
       20031.35544835])

In [177]:
pd.DataFrame({'Prognoz':predicted_data, 'Real baxosi': test_label})

,Prognoz,Real baxosi
4664,43803.506615,30800
6895,98108.865235,120000
6079,23549.377611,29500
2172,43537.240543,33505
7111,20031.355448,25500


In [178]:
test_set

,district,rooms,size,level,max_levels,price
2301,Яшнободский,3,86.0,4,9,89000
6126,Учтепинский,3,70.0,2,5,43500
2149,Яккасарайский,3,90.0,5,5,45000
5594,Чиланзарский,1,36.0,2,4,24700
6566,Учтепинский,4,100.0,1,4,87000
...,...,...,...,...,...,...
6593,Шайхантахурский,3,80.0,6,9,48000
4089,Шайхантахурский,6,500.0,2,2,1800
441,Чиланзарский,1,36.0,2,5,28500
5639,Шайхантахурский,3,70.0,6,7,60000


In [180]:
X_test = test_set.drop('price', axis=1)
X_test

,district,rooms,size,level,max_levels
2301,Яшнободский,3,86.0,4,9
6126,Учтепинский,3,70.0,2,5
2149,Яккасарайский,3,90.0,5,5
5594,Чиланзарский,1,36.0,2,4
6566,Учтепинский,4,100.0,1,4
...,...,...,...,...,...
6593,Шайхантахурский,3,80.0,6,9
4089,Шайхантахурский,6,500.0,2,2
441,Чиланзарский,1,36.0,2,5
5639,Шайхантахурский,3,70.0,6,7


In [182]:
y_test = test_set['price'].copy()
y_test

2301    89000
6126    43500
2149    45000
5594    24700
6566    87000
        ...  
6593    48000
4089     1800
441     28500
5639    60000
2538    42000
Name: price, Length: 1493, dtype: int64

In [183]:
X_test_prepared = full_pipeline.transform(X_test)

In [184]:
y_predicted = LR_model.predict(X_test_prepared)


In [186]:
from sklearn.metrics import mean_squared_error
import numpy as np
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

1366742.755611274


In [187]:
from sklearn.tree import DecisionTreeRegressor
Tree_model = DecisionTreeRegressor()
Tree_model.fit(X_prepared, y)

DecisionTreeRegressor()

In [188]:
y_predicted = Tree_model.predict(X_test_prepared)


In [189]:
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

1366793.4505425717


In [190]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [191]:
y_predicted = RF_model.predict(X_test_prepared)
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

1370535.5837398914


In [193]:
X = df.drop("price", axis=1)
y = df["price"].copy()

X_prepared = full_pipeline.transform(X)

In [195]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Std.dev:", scores.std())

In [196]:
from sklearn.model_selection import cross_val_score



In [197]:
scores = cross_val_score(LR_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)

desicion tree

In [198]:
display_scores(LR_rmse_scores)

Scores: [  72555.73963232   46818.73106055  144429.71285792   42474.71976401
   48253.58258335   43527.84943424   52085.7886351  1901388.20302356
  166777.44610516  650092.90111387]
Mean: 316840.4674210096
Std.dev: 556676.7608983462


random forest

In [199]:
scores = cross_val_score(RF_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [  77315.12354168  148890.17577362  315195.36244182   79899.17680032
  269345.36663014  140993.67586632  202175.34180228 1903225.62901318
  185987.82342307  648121.9328793 ]
Mean: 397114.9608171734
Std.dev: 526032.062135558


modelni saqlash

In [200]:
import pickle

filename = 'RF_model.pkl' # faylga istalgan nom beramiz
with open(filename, 'wb') as file:
    pickle.dump(RF_model, file)

In [201]:
with open(filename, 'rb') as file:
    model = pickle.load(file)

In [202]:
scores = cross_val_score(model, X_prepared, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [ 116339.46217094  198699.66685036  193665.68133337 1347142.12116148
  511643.28846521]
Mean: 473498.04399627337
Std.dev: 457403.8882844006
